In [1]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import warnings
import time
warnings.filterwarnings('ignore')


In [2]:
class Auto_Inviter():
    def __init__(self,user_profile_path, chromedriver_path):
        '''
        load the chrome profile for auto login and create the webdriver
        '''
        options = webdriver.ChromeOptions()
        options.add_argument('--user-data-dir=' + user_profile_path) 
        options.add_argument("start-maximized")
        options.add_argument("disable-infobars")
        options.add_argument("--disable-extensions")
        self.driver = webdriver.Chrome(chromedriver_path,chrome_options=options)
        self.driver.maximize_window()

    def open_website(self, url, company_name):
        '''
        open the website according to the url
        '''
        self.driver.get(url)
        self.company = company_name

    def get_user_names(self):
        '''
        get the users' names on the searching page
        '''
        WebDriverWait(self.driver, 5, 0.5).until(EC.visibility_of_element_located((By.XPATH, "//*[@class='visually-hidden']")))
        titles = self.driver.find_elements_by_class_name('visually-hidden')
        names = []
        for title in titles:
            if 'View' in title.text:
                name = title.text.replace('\n','').replace(('View '),'').replace(('’s profile'),'')
                names.append(name)
        return names

    def open_connector_pagination(self, target):
        '''
        open the new page according to the user name and switch the control to new page
        Furture work: for other os the command to open a new page is different now only test on macos
        '''
        ActionChains(self.driver).key_down(Keys.COMMAND).perform()
        target.click()
        ActionChains(self.driver).key_up(Keys.COMMAND).perform()
        self.driver.switch_to.window(self.driver.window_handles[1])

    def close_connector_pagination(self):
        '''
        close the new page and return control back to searching page
        '''
        self.driver.close()
        self.driver.switch_to.window(self.driver.window_handles[0])

    def next_page(self):
        '''
        click the 'Next' button to new page and new users
        '''
        WebDriverWait(self.driver, 10, 0.5).until(EC.visibility_of_element_located((By.XPATH, "//span[@class='artdeco-button__text' and text()='Next']"))).click()

    def send_msg(self, name, msg):
        '''
        send the msg to connectors
        there are two differnt types of forms
        1. you can't directly click the connect button on the website (you have to click more->connection->.......)
        2. you can directly click the connect button
        in this fuction I deal with both situations
        '''
        msg = 'Hi '+name.split(' ')[0]+',\n' + msg
        if len(msg) > 300:
            print('The message is too long and there will be some missing words!!!')

        self.driver.switch_to.window(self.driver.window_handles[1])
        try:
            self.driver.find_elements_by_xpath('//button[@aria-label="More actions"]')[1].click()
            self.driver.find_elements_by_xpath("//span[@class='display-flex t-normal flex-1' and text()='Connect']")[1].click()
            self.driver.find_elements_by_xpath("//span[@class='artdeco-button__text' and text()='Connect']")[0].click()
        except:
            self.driver.find_elements_by_xpath("//span[@class='artdeco-button__text' and text()='Connect']")[1].click()
        self.driver.find_element_by_xpath('//button[normalize-space()="Add a note"]').click()
        self.driver.find_element_by_id('custom-message').send_keys(msg)
        self.driver.find_element_by_xpath("//span[@class='artdeco-button__text' and text()='Send']").click()
        print('sucess')
        #time.sleep(3)

    def ask_for_referral(self, msg, num):
        '''
        reach out connectors and ask for referrals according to the num count
        sometimes the website will show 'Retry search' so I use a expection to handle this situation
        '''
        i = 0
        while True:
            try:
                WebDriverWait(self.driver, 5, 0.5).until(EC.visibility_of_element_located((By.XPATH, "//*[@class='app-aware-link']")))
            except:
                self.driver.find_element_by_xpath("//span[@class='artdeco-button__text' and text()='Retry search']").click()
                print('renew')
                continue
            links = self.driver.find_elements_by_class_name('app-aware-link')
            names = self.get_user_names()
            for link in links:
                name = link.text.split('\n')[0]
                if name in names:
                    names.remove(name)
                    i+=1
                    print(i,": ",name)
                    '''self.open_connector_pagination(link)
                    self.send_msg(name,msg)
                    self.close_connector_pagination()'''
                if i == num:
                    return
                    
            # go to the bottom of the page to find out the 'Next' button
            self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            self.next_page()

In [3]:
company_url = 'https://www.linkedin.com/search/results/people/?keywords=google&origin=SWITCH_SEARCH_VERTICAL&sid=OlG'
company_name = 'google'
num = 10
msg ='My name is Alex, a UCSD MSCS student. I found your profile on Linkedln. Moreover, currently I\'m looking for internship in the USA and I know that you work at '+company_name+' and I\'m impressed by you experience. I\'m just wondering can I get more connection or referral from you.'

# One time setting
user_profile_path = '/Users/yanghangwu/Library/Application Support/Google/Chrome/Profile 3'
chromedriver_path = './chromedriver98'

Inviter = Auto_Inviter(user_profile_path, chromedriver_path)
Inviter.open_website(company_url, company_name)
Inviter.ask_for_referral(msg, num)

1 :  Ziyang Wang
2 :  Shun-Yao (Gary) Shih
3 :  Shikan Lian
4 :  Katie Bromley
5 :  Menghao Zhao
6 :  Chu-Ling Ko
7 :  Chun-Hsiang Wang
8 :  Jinjun Ding
9 :  Yuxi Sun
10 :  Xuan Yang
